## Loading necessary packages

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
from __future__ import print_function
from statsmodels.compat import lzip
import matplotlib.pyplot as plt
import numpy as np
import sklearn as sk
from sklearn.linear_model import LinearRegression

In [2]:
from statsmodels.graphics.regressionplots import plot_fit
from statsmodels.graphics.regressionplots import plot_leverage_resid2
from statsmodels.graphics.regressionplots import plot_regress_exog
from sklearn import linear_model
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import Ridge
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model
from sklearn.pipeline import make_pipeline

In [3]:
from sklearn.metrics import r2_score, mean_squared_error
linear_regression = linear_model.LinearRegression(normalize=False, fit_intercept=True)
def r2_est(X,y):
    return r2_score(y, linear_regression.fit(X,y).predict(X))
def r2_est_two(X,y, X_new, y_new):
    return r2_score(y_new, linear_regression.fit(X,y).predict(X_new))

In [4]:
from sklearn.preprocessing import LabelEncoder, LabelBinarizer
from sklearn.decomposition import PCA
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.cross_validation import cross_val_score, KFold, StratifiedKFold
from sklearn.metrics import make_scorer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.feature_selection import RFECV
from sklearn.cross_validation import KFold
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import Lasso
from sklearn.grid_search import RandomizedSearchCV
from scipy.stats import expon

In [5]:
from sklearn.preprocessing import LabelEncoder, LabelBinarizer
label_enc = LabelEncoder()
label_bin = LabelBinarizer()

## Data Loading and Preparation (time series)

In [9]:
cal = pd.read_csv('Datasources/inside_airbnb/calendar.csv')

In [10]:
cal['price'] = cal['price'].astype(str).map(lambda x: x.lstrip('$'))
cal['price'] = cal['price'].apply(pd.to_numeric, errors='coerce')
cal['price'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')

In [11]:
cal['date'] = pd.to_datetime(cal['date'])
cal['month'] = cal['date'].apply(lambda x: x.month)
cal['day'] = cal['date'].apply(lambda x: x.day)

In [12]:
cal['day_of_week'] = cal['date'].dt.weekday_name

In [13]:
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar

In [14]:
cl = calendar()
holidays = cl.holidays(start=cal['date'].min(), end=cal['date'].max())

In [15]:
cal['holiday'] = cal['date'].isin(holidays)

In [16]:
c = cal.loc[cal.available!='f']
c = c[['listing_id','date','price','month','day_of_week','holiday']]

In [17]:
c_hol = c[c['holiday']==True]
c_wke = c[(c['holiday']==False)&((c['day_of_week']=='Sunday')|(c['day_of_week']=='Saturday'))]
c_wkd = c[(~c.isin(c_hol['date']))&(~c.isin(c_wke['date']))]

In [18]:
mean_price_hol = c_hol.groupby(by='listing_id')['price'].mean()
mean_price_wke = c_wke.groupby(by='listing_id')['price'].mean()
mean_price_wkd = c_wkd.groupby(by='listing_id')['price'].mean()

In [19]:
mean_price_hol = pd.DataFrame(mean_price_hol)
mean_price_wke = pd.DataFrame(mean_price_wke)
mean_price_wkd = pd.DataFrame(mean_price_wkd)

In [20]:
mean_price_hol = mean_price_hol.reset_index()
mean_price_wke = mean_price_wke.reset_index()
mean_price_wkd = mean_price_wkd.reset_index()

In [21]:
listings = pd.read_csv('Datasources/inside_airbnb/listings.csv')

In [22]:
master_hol = listings.merge(mean_price_hol, how='inner', left_on='id', right_on='listing_id')
master_wke = listings.merge(mean_price_wke, how='inner', left_on='id', right_on='listing_id')
master_wkd = listings.merge(mean_price_wkd, how='inner', left_on='id', right_on='listing_id')

## Feature Creation (time series)

### Features Created in regression

In [23]:
master_hol['host_response_rate'] = master_hol['host_response_rate'].astype(str).map(lambda x: x.rstrip("%"))
master_hol['host_response_rate'] = master_hol['host_response_rate'].apply(pd.to_numeric, errors='coerce')
master_hol['host_response_rate'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
master_hol['host_acceptance_rate'] = master_hol['host_acceptance_rate'].astype(str).map(lambda x: x.rstrip("%"))
master_hol['host_acceptance_rate'] = master_hol['host_acceptance_rate'].apply(pd.to_numeric, errors='coerce')
master_hol['host_acceptance_rate'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
master_hol['extra_people'] = master_hol['extra_people'].astype(str).map(lambda x: x.lstrip('$'))
master_hol['extra_people'] = master_hol['extra_people'].apply(pd.to_numeric, errors='coerce')
master_hol['extra_people'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
master_hol['weekly_price'] = master_hol['weekly_price'].astype(str).map(lambda x: x.lstrip('$'))
master_hol['weekly_price'] = master_hol['weekly_price'].apply(pd.to_numeric, errors='coerce')
master_hol['weekly_price'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
master_hol['monthly_price'] = master_hol['monthly_price'].astype(str).map(lambda x: x.lstrip('$'))
master_hol['monthly_price'] = master_hol['monthly_price'].apply(pd.to_numeric, errors='coerce')
master_hol['monthly_price'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
master_hol['security_deposit'] = master_hol['security_deposit'].astype(str).map(lambda x: x.lstrip('$'))
master_hol['security_deposit'] = master_hol['security_deposit'].apply(pd.to_numeric, errors='coerce')
master_hol['security_deposit'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
master_hol['cleaning_fee'] = master_hol['cleaning_fee'].astype(str).map(lambda x: x.lstrip('$'))
master_hol['cleaning_fee'] = master_hol['cleaning_fee'].apply(pd.to_numeric, errors='coerce')
master_hol['cleaning_fee'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')

In [24]:
master_wke['host_response_rate'] = master_wke['host_response_rate'].astype(str).map(lambda x: x.rstrip("%"))
master_wke['host_response_rate'] = master_wke['host_response_rate'].apply(pd.to_numeric, errors='coerce')
master_wke['host_response_rate'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
master_wke['host_acceptance_rate'] = master_wke['host_acceptance_rate'].astype(str).map(lambda x: x.rstrip("%"))
master_wke['host_acceptance_rate'] = master_wke['host_acceptance_rate'].apply(pd.to_numeric, errors='coerce')
master_wke['host_acceptance_rate'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
master_wke['extra_people'] = master_wke['extra_people'].astype(str).map(lambda x: x.lstrip('$'))
master_wke['extra_people'] = master_wke['extra_people'].apply(pd.to_numeric, errors='coerce')
master_wke['extra_people'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
master_wke['weekly_price'] = master_wke['weekly_price'].astype(str).map(lambda x: x.lstrip('$'))
master_wke['weekly_price'] = master_wke['weekly_price'].apply(pd.to_numeric, errors='coerce')
master_wke['weekly_price'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
master_wke['monthly_price'] = master_wke['monthly_price'].astype(str).map(lambda x: x.lstrip('$'))
master_wke['monthly_price'] = master_wke['monthly_price'].apply(pd.to_numeric, errors='coerce')
master_wke['monthly_price'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
master_wke['security_deposit'] = master_wke['security_deposit'].astype(str).map(lambda x: x.lstrip('$'))
master_wke['security_deposit'] = master_wke['security_deposit'].apply(pd.to_numeric, errors='coerce')
master_wke['security_deposit'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
master_wke['cleaning_fee'] = master_wke['cleaning_fee'].astype(str).map(lambda x: x.lstrip('$'))
master_wke['cleaning_fee'] = master_wke['cleaning_fee'].apply(pd.to_numeric, errors='coerce')
master_wke['cleaning_fee'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')

In [25]:
master_wkd['host_response_rate'] = master_wkd['host_response_rate'].astype(str).map(lambda x: x.rstrip("%"))
master_wkd['host_response_rate'] = master_wkd['host_response_rate'].apply(pd.to_numeric, errors='coerce')
master_wkd['host_response_rate'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
master_wkd['host_acceptance_rate'] = master_wkd['host_acceptance_rate'].astype(str).map(lambda x: x.rstrip("%"))
master_wkd['host_acceptance_rate'] = master_wkd['host_acceptance_rate'].apply(pd.to_numeric, errors='coerce')
master_wkd['host_acceptance_rate'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
master_wkd['extra_people'] = master_wkd['extra_people'].astype(str).map(lambda x: x.lstrip('$'))
master_wkd['extra_people'] = master_wkd['extra_people'].apply(pd.to_numeric, errors='coerce')
master_wkd['extra_people'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
master_wkd['weekly_price'] = master_wkd['weekly_price'].astype(str).map(lambda x: x.lstrip('$'))
master_wkd['weekly_price'] = master_wkd['weekly_price'].apply(pd.to_numeric, errors='coerce')
master_wkd['weekly_price'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
master_wkd['monthly_price'] = master_wkd['monthly_price'].astype(str).map(lambda x: x.lstrip('$'))
master_wkd['monthly_price'] = master_wkd['monthly_price'].apply(pd.to_numeric, errors='coerce')
master_wkd['monthly_price'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
master_wkd['security_deposit'] = master_wkd['security_deposit'].astype(str).map(lambda x: x.lstrip('$'))
master_wkd['security_deposit'] = master_wkd['security_deposit'].apply(pd.to_numeric, errors='coerce')
master_wkd['security_deposit'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
master_wkd['cleaning_fee'] = master_wkd['cleaning_fee'].astype(str).map(lambda x: x.lstrip('$'))
master_wkd['cleaning_fee'] = master_wkd['cleaning_fee'].apply(pd.to_numeric, errors='coerce')
master_wkd['cleaning_fee'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')

In [26]:
master_hol['ones'] = np.ones(len(master_hol))
master_wke['ones'] = np.ones(len(master_wke))
master_wkd['ones'] = np.ones(len(master_wkd))

### Features created from amenities

In [27]:
def string_to_set(x):
    c = set()
    for w in x[1:-1].split(","):
        c.add(w)
        
    return c

In [28]:
master_hol['amenities_set'] = master_hol['amenities'].map(string_to_set)
master_wke['amenities_set'] = master_wke['amenities'].map(string_to_set)
master_wkd['amenities_set'] = master_wkd['amenities'].map(string_to_set)

In [29]:
def has_amenity(x, amen_):
    if amen_ in x:
        return 1
    pass

In [30]:
all_amenities = set()

for idx in listings['amenities'].map(string_to_set).index:
    all_amenities = all_amenities.union(listings['amenities'].map(string_to_set)[idx])

all_amenities

{'',
 '"24-Hour Check-in"',
 '"Air Conditioning"',
 '"Buzzer/Wireless Intercom"',
 '"Cable TV"',
 '"Carbon Monoxide Detector"',
 '"Elevator in Building"',
 '"Family/Kid Friendly"',
 '"Fire Extinguisher"',
 '"First Aid Kit"',
 '"Free Parking on Premises"',
 '"Hair Dryer"',
 '"Hot Tub"',
 '"Indoor Fireplace"',
 '"Laptop Friendly Workspace"',
 '"Lock on Bedroom Door"',
 '"Other pet(s)"',
 '"Pets Allowed"',
 '"Pets live on this property"',
 '"Safety Card"',
 '"Smoke Detector"',
 '"Smoking Allowed"',
 '"Suitable for Events"',
 '"Wheelchair Accessible"',
 '"Wireless Internet"',
 '"translation missing: en.hosting_amenity_49"',
 '"translation missing: en.hosting_amenity_50"',
 'Breakfast',
 'Cat(s)',
 'Dog(s)',
 'Doorman',
 'Dryer',
 'Essentials',
 'Gym',
 'Hangers',
 'Heating',
 'Internet',
 'Iron',
 'Kitchen',
 'Pool',
 'Shampoo',
 'TV',
 'Washer'}

In [31]:
for amen in all_amenities:
    master_hol['has' + amen] = 0
    master_wke['has' + amen] = 0
    master_wkd['has' + amen] = 0
    master_hol['has' + amen] = master_hol['amenities_set'].map(lambda x: has_amenity(x, amen))
    master_wke['has' + amen] = master_wke['amenities_set'].map(lambda x: has_amenity(x, amen))
    master_wkd['has' + amen] = master_wkd['amenities_set'].map(lambda x: has_amenity(x, amen))

In [32]:
has_amenties_list = []
for amen in all_amenities:
    has_amenties_list.append('has' + amen)

In [33]:
master_hol[has_amenties_list] = master_hol[has_amenties_list].fillna(0)
master_wke[has_amenties_list] = master_wke[has_amenties_list].fillna(0)
master_wkd[has_amenties_list] = master_wkd[has_amenties_list].fillna(0)

### Preliminary Data Cleaning and Column Dropping

In [34]:
X_hol = master_hol.select_dtypes(include=['float64', 'int64'])
X_wke = master_wke.select_dtypes(include=['float64', 'int64'])
X_wkd = master_wkd.select_dtypes(include=['float64', 'int64'])

In [35]:
X_hol.isnull().sum()

id                                                    0
scrape_id                                             0
host_id                                               0
host_response_rate                                  297
host_acceptance_rate                                469
host_listings_count                                   0
host_total_listings_count                             0
neighbourhood_group_cleansed                       5208
zipcode                                              77
latitude                                              0
longitude                                             0
accommodates                                          0
bathrooms                                            10
bedrooms                                              3
beds                                                  4
square_feet                                        5112
weekly_price                                       4157
monthly_price                                   

In [38]:
X_hol = X_hol.drop(['neighbourhood_group_cleansed','square_feet','has_availability','license','weekly_price','monthly_price'], axis = 1)
X_wke = X_wke.drop(['neighbourhood_group_cleansed','square_feet','has_availability','license','weekly_price','monthly_price'], axis = 1)
X_wkd = X_wkd.drop(['neighbourhood_group_cleansed','square_feet','has_availability','license','weekly_price','monthly_price'], axis = 1)

In [39]:
X_hol = X_hol.fillna(value=X_hol.mean())
X_wke = X_wke.fillna(value=X_wke.mean())
X_wkd = X_wkd.fillna(value=X_wkd.mean())

In [41]:
X_hol = X_hol.drop(['id','listing_id','host_id','scrape_id'], axis = 1)
X_wke = X_wke.drop(['id','listing_id','host_id','scrape_id'], axis = 1)
X_wkd = X_wkd.drop(['id','listing_id','host_id','scrape_id'], axis = 1)

## Including Encoded and Binarized Variables

#### Encoded Variables

In [42]:
X_hol['policy_enc'] = label_enc.fit_transform(master_hol.cancellation_policy)
master_hol['host_response_time'] = master_hol['host_response_time'].astype(str)
X_hol['host_resp_time_enc'] = label_enc.fit_transform(master_hol.host_response_time)
master_hol['calendar_updated'] = master_hol['calendar_updated'].astype(str)
X_hol['calendar_updated_enc'] = label_enc.fit_transform(master_hol.calendar_updated)
master_hol['bed_type'] = master_hol['bed_type'].astype(str)
X_hol['bed_type_enc'] = label_enc.fit_transform(master_hol.bed_type)
master_hol['jurisdiction_names'] = master_hol['jurisdiction_names'].astype(str)
X_hol['jurisdiction_names_enc'] = label_enc.fit_transform(master_hol.jurisdiction_names)
master_hol['zipcode'] = master_hol['zipcode'].astype(str)
X_hol['zipcode_enc'] = label_enc.fit_transform(master_hol.zipcode)

In [43]:
X_wke['policy_enc'] = label_enc.fit_transform(master_wke.cancellation_policy)
master_wke['host_response_time'] = master_wke['host_response_time'].astype(str)
X_wke['host_resp_time_enc'] = label_enc.fit_transform(master_wke.host_response_time)
master_wke['calendar_updated'] = master_wke['calendar_updated'].astype(str)
X_wke['calendar_updated_enc'] = label_enc.fit_transform(master_wke.calendar_updated)
master_wke['bed_type'] = master_wke['bed_type'].astype(str)
X_wke['bed_type_enc'] = label_enc.fit_transform(master_wke.bed_type)
master_wke['jurisdiction_names'] = master_wke['jurisdiction_names'].astype(str)
X_wke['jurisdiction_names_enc'] = label_enc.fit_transform(master_wke.jurisdiction_names)
master_wke['zipcode'] = master_wke['zipcode'].astype(str)
X_wke['zipcode_enc'] = label_enc.fit_transform(master_wke.zipcode)

In [44]:
X_wkd['policy_enc'] = label_enc.fit_transform(master_wkd.cancellation_policy)
master_wkd['host_response_time'] = master_wkd['host_response_time'].astype(str)
X_wkd['host_resp_time_enc'] = label_enc.fit_transform(master_wkd.host_response_time)
master_wkd['calendar_updated'] = master_wkd['calendar_updated'].astype(str)
X_wkd['calendar_updated_enc'] = label_enc.fit_transform(master_wkd.calendar_updated)
master_wkd['bed_type'] = master_wkd['bed_type'].astype(str)
X_wkd['bed_type_enc'] = label_enc.fit_transform(master_wkd.bed_type)
master_wkd['jurisdiction_names'] = master_wkd['jurisdiction_names'].astype(str)
X_wkd['jurisdiction_names_enc'] = label_enc.fit_transform(master_wkd.jurisdiction_names)
master_wkd['zipcode'] = master_wkd['zipcode'].astype(str)
X_wkd['zipcode_enc'] = label_enc.fit_transform(master_wkd.zipcode)

#### Binarized Variables

In [45]:
X_hol['host_is_superhost'] = label_bin.fit_transform(master_hol.host_is_superhost)
X_hol['is_location_exact'] = label_bin.fit_transform(master_hol.is_location_exact)
X_hol['host_profile_pic'] = label_bin.fit_transform(master_hol.host_has_profile_pic)
X_hol['host_identity_verified'] = label_bin.fit_transform(master_hol.host_identity_verified)
X_hol['instant_bookable'] = label_bin.fit_transform(master_hol.instant_bookable)
X_hol['require_guest_profile_picture'] = label_bin.fit_transform(master_hol.require_guest_profile_picture)
X_hol['require_guest_phone_verification'] = label_bin.fit_transform(master_hol.require_guest_phone_verification)

In [46]:
X_wke['host_is_superhost'] = label_bin.fit_transform(master_wke.host_is_superhost)
X_wke['is_location_exact'] = label_bin.fit_transform(master_wke.is_location_exact)
X_wke['host_profile_pic'] = label_bin.fit_transform(master_wke.host_has_profile_pic)
X_wke['host_identity_verified'] = label_bin.fit_transform(master_wke.host_identity_verified)
X_wke['instant_bookable'] = label_bin.fit_transform(master_wke.instant_bookable)
X_wke['require_guest_profile_picture'] = label_bin.fit_transform(master_wke.require_guest_profile_picture)
X_wke['require_guest_phone_verification'] = label_bin.fit_transform(master_wke.require_guest_phone_verification)

In [47]:
X_wkd['host_is_superhost'] = label_bin.fit_transform(master_wkd.host_is_superhost)
X_wkd['is_location_exact'] = label_bin.fit_transform(master_wkd.is_location_exact)
X_wkd['host_profile_pic'] = label_bin.fit_transform(master_wkd.host_has_profile_pic)
X_wkd['host_identity_verified'] = label_bin.fit_transform(master_wkd.host_identity_verified)
X_wkd['instant_bookable'] = label_bin.fit_transform(master_wkd.instant_bookable)
X_wkd['require_guest_profile_picture'] = label_bin.fit_transform(master_wkd.require_guest_profile_picture)
X_wkd['require_guest_phone_verification'] = label_bin.fit_transform(master_wkd.require_guest_phone_verification)

## Including Interactions

In [48]:
X_hol['bathrooms*cleaning_fee'] = X_hol['bathrooms'] * X_hol['cleaning_fee']
X_hol['bathrooms*guests_included'] = X_hol['bathrooms'] * X_hol['guests_included']
X_hol['bathrooms*addommodates'] = X_hol['accommodates'] * X_hol['bathrooms']
X_hol['accomodates*cleaning_fee'] = X_hol['accommodates'] * X_hol['cleaning_fee']

In [49]:
X_wke['bathrooms*cleaning_fee'] = X_wke['bathrooms'] * X_wke['cleaning_fee']
X_wke['bathrooms*guests_included'] = X_wke['bathrooms'] * X_wke['guests_included']
X_wke['bathrooms*addommodates'] = X_wke['accommodates'] * X_wke['bathrooms']
X_wke['accomodates*cleaning_fee'] = X_wke['accommodates'] * X_wke['cleaning_fee']

In [50]:
X_wkd['bathrooms*cleaning_fee'] = X_wkd['bathrooms'] * X_wkd['cleaning_fee']
X_wkd['bathrooms*guests_included'] = X_wkd['bathrooms'] * X_wkd['guests_included']
X_wkd['bathrooms*addommodates'] = X_wkd['accommodates'] * X_wkd['bathrooms']
X_wkd['accomodates*cleaning_fee'] = X_wkd['accommodates'] * X_wkd['cleaning_fee']

## Target and Predictors dataframe creation

In [51]:
target_hol = X_hol['price_y']
target_wke = X_wke['price_y']
target_wkd = X_wkd['price_y']

In [52]:
X_hol = X_hol.drop('price_y', axis = 1)
X_wke = X_wke.drop('price_y', axis = 1)
X_wkd = X_wkd.drop('price_y', axis = 1)

### Whole Target and Predictors dataframe creation

In [53]:
target_whole = reduce(lambda x, y: x.combine_first(y), [target_hol, target_wke, target_wkd])
X_whole = reduce(lambda x, y: x.combine_first(y), [X_hol, X_wke, X_wkd])

## Modeling (time series)

### Model 3

#### Holidays

In [69]:
X_train, X_test, y_train, y_test = train_test_split(X_hol, target_hol, test_size=.30, random_state=1)
X_train_two, X_val, y_train_two, y_val = train_test_split(X_train, y_train, test_size=.30, random_state=1)

##### Linear Model

In [56]:
print('R2 for validation is ' + str(r2_est_two(X_train_two, y_train_two, X_val, y_val)))
print('Mean square error for validation is ' + str(mean_squared_error(linear_regression.fit(X_train_two, y_train_two).predict(X_val), y_val)))
print('Mean absolute error for validation is ' + str(mean_absolute_error(linear_regression.fit(X_train_two, y_train_two).predict(X_val), y_val)))
print('R2 is ' + str(r2_est_two(X_train, y_train, X_test, y_test)))
print('Mean square error is ' + str(mean_squared_error(linear_regression.fit(X_train, y_train).predict(X_test), y_test)))
print('Mean absolute error is ' + str(mean_absolute_error(linear_regression.fit(X_train, y_train).predict(X_test), y_test)))

R2 for validation is 0.667332578188
Mean square error for validation is 8587.70808747
Mean absolute error for validation is 61.9781239998
R2 is -1.61958634403
Mean square error is 66439.3511622
Mean absolute error is 67.4951681112


##### Quadratic Model

In [70]:
create_poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
poly = make_pipeline(create_poly, StandardScaler(), linear_regression)
model = poly.fit(X_train_two, y_train_two)
print('R^2 for validation is ' + str(r2_score(model.predict(X_val), y_val)))
print('Mean square error for validation is ' + str(mean_squared_error(model.predict(X_val), y_val)))
print('Mean absolute error for validation is ' + str(mean_absolute_error(model.predict(X_val), y_val)))
model_final = poly.fit(X_train, y_train)
print('R^2 for is ' + str(r2_score(model.predict(X_test), y_test)))
print('Mean square error is ' + str(mean_squared_error(model.predict(X_test), y_test)))
print('Mean absolute error is ' + str(mean_absolute_error(model.predict(X_test), y_test)))

R^2 for validation is 0.0071325269226
Mean square error for validation is 10343890.6773
Mean absolute error for validation is 1888.96237253
R^2 for is -0.00229576787193
Mean square error is 23957406.5521
Mean absolute error is 1824.57668503


#### Weekends

In [71]:
X_train, X_test, y_train, y_test = train_test_split(X_wke, target_wke, test_size=.30, random_state=1)
X_train_two, X_val, y_train_two, y_val = train_test_split(X_train, y_train, test_size=.30, random_state=1)

##### Linear Model

In [60]:
print('R2 for validation is ' + str(r2_est_two(X_train_two, y_train_two, X_val, y_val)))
print('Mean square error for validation is ' + str(mean_squared_error(linear_regression.fit(X_train_two, y_train_two).predict(X_val), y_val)))
print('Mean absolute error for validation is ' + str(mean_absolute_error(linear_regression.fit(X_train_two, y_train_two).predict(X_val), y_val)))
print('R2 is ' + str(r2_est_two(X_train, y_train, X_test, y_test)))
print('Mean square error is ' + str(mean_squared_error(linear_regression.fit(X_train, y_train).predict(X_test), y_test)))
print('Mean absolute error is ' + str(mean_absolute_error(linear_regression.fit(X_train, y_train).predict(X_test), y_test)))

R2 for validation is 0.613288860204
Mean square error for validation is 11289.0120014
Mean absolute error for validation is 68.6589471424
R2 is -1.92156598143
Mean square error is 77332.8416202
Mean absolute error is 73.7650414509


##### Quadratic Model 

In [72]:
create_poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
poly = make_pipeline(create_poly, StandardScaler(), linear_regression)
model = poly.fit(X_train_two, y_train_two)
print('R^2 for validation is ' + str(r2_score(model.predict(X_val), y_val)))
print('Mean square error for validation is ' + str(mean_squared_error(model.predict(X_val), y_val)))
print('Mean absolute error for validation is ' + str(mean_absolute_error(model.predict(X_val), y_val)))
model_final = poly.fit(X_train, y_train)
print('R^2 for is ' + str(r2_score(model.predict(X_test), y_test)))
print('Mean square error is ' + str(mean_squared_error(model.predict(X_test), y_test)))
print('Mean absolute error is ' + str(mean_absolute_error(model.predict(X_test), y_test)))

R^2 for validation is 0.00182263532958
Mean square error for validation is 1851395.35407
Mean absolute error for validation is 820.245818553
R^2 for is -0.0021854146768
Mean square error is 6.26746507071e+24
Mean absolute error is 277544084982.0


#### Weekdays 

In [73]:
X_train, X_test, y_train, y_test = train_test_split(X_wkd, target_wkd, test_size=.30, random_state=1)
X_train_two, X_val, y_train_two, y_val = train_test_split(X_train, y_train, test_size=.30, random_state=1)

##### Linear Model

In [64]:
print('R2 for validation is ' + str(r2_est_two(X_train_two, y_train_two, X_val, y_val)))
print('Mean square error for validation is ' + str(mean_squared_error(linear_regression.fit(X_train_two, y_train_two).predict(X_val), y_val)))
print('Mean absolute error for validation is ' + str(mean_absolute_error(linear_regression.fit(X_train_two, y_train_two).predict(X_val), y_val)))
print('R2 is ' + str(r2_est_two(X_train, y_train, X_test, y_test)))
print('Mean square error is ' + str(mean_squared_error(linear_regression.fit(X_train, y_train).predict(X_test), y_test)))
print('Mean absolute error is ' + str(mean_absolute_error(linear_regression.fit(X_train, y_train).predict(X_test), y_test)))

R2 for validation is 0.607959811208
Mean square error for validation is 9979.06322611
Mean absolute error for validation is 66.427615843
R2 is 0.640284864283
Mean square error is 10317.1451407
Mean absolute error is 65.821378261


##### Quadratic Model

In [74]:
create_poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
poly = make_pipeline(create_poly, StandardScaler(), linear_regression)
model = poly.fit(X_train_two, y_train_two)
print('R^2 for validation is ' + str(r2_score(model.predict(X_val), y_val)))
print('Mean square error for validation is ' + str(mean_squared_error(model.predict(X_val), y_val)))
print('Mean absolute error for validation is ' + str(mean_absolute_error(model.predict(X_val), y_val)))
model_final = poly.fit(X_train, y_train)
print('R^2 for is ' + str(r2_score(model.predict(X_test), y_test)))
print('Mean square error is ' + str(mean_squared_error(model.predict(X_test), y_test)))
print('Mean absolute error is ' + str(mean_absolute_error(model.predict(X_test), y_test)))

R^2 for validation is 0.0259912623834
Mean square error for validation is 431557.97775
Mean absolute error for validation is 381.815474329
R^2 for is -0.000272081381259
Mean square error is 1.09943716594e+24
Mean absolute error is 115709905216.0


#### As a whole

In [75]:
X_train, X_test, y_train, y_test = train_test_split(X_whole, target_whole, test_size=.30, random_state=1)
X_train_two, X_val, y_train_two, y_val = train_test_split(X_train, y_train, test_size=.30, random_state=1)

##### Linear Model

In [67]:
print('R2 for validation is ' + str(r2_est_two(X_train_two, y_train_two, X_val, y_val)))
print('Mean square error for validation is ' + str(mean_squared_error(linear_regression.fit(X_train_two, y_train_two).predict(X_val), y_val)))
print('Mean absolute error for validation is ' + str(mean_absolute_error(linear_regression.fit(X_train_two, y_train_two).predict(X_val), y_val)))
print('R2 is ' + str(r2_est_two(X_train, y_train, X_test, y_test)))
print('Mean square error is ' + str(mean_squared_error(linear_regression.fit(X_train, y_train).predict(X_test), y_test)))
print('Mean absolute error is ' + str(mean_absolute_error(linear_regression.fit(X_train, y_train).predict(X_test), y_test)))

R2 for validation is 0.571326041779
Mean square error for validation is 9745.66385281
Mean absolute error for validation is 62.2131388404
R2 is 0.63589271336
Mean square error is 9457.98660233
Mean absolute error is 60.9864236218


##### Quadratic Model

In [76]:
create_poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
poly = make_pipeline(create_poly, StandardScaler(), linear_regression)
model = poly.fit(X_train_two, y_train_two)
print('R^2 for validation is ' + str(r2_score(model.predict(X_val), y_val)))
print('Mean square error for validation is ' + str(mean_squared_error(model.predict(X_val), y_val)))
print('Mean absolute error for validation is ' + str(mean_absolute_error(model.predict(X_val), y_val)))
model_final = poly.fit(X_train, y_train)
print('R^2 for is ' + str(r2_score(model.predict(X_test), y_test)))
print('Mean square error is ' + str(mean_squared_error(model.predict(X_test), y_test)))
print('Mean absolute error is ' + str(mean_absolute_error(model.predict(X_test), y_test)))

R^2 for validation is -0.000883201004719
Mean square error for validation is 5.28121228395e+13
Mean absolute error for validation is 1508073.97021
R^2 for is -0.000455285076517
Mean square error is 1.07677213861e+23
Mean absolute error is 38650090287.9
